In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # 输入一个字符串，将它编码为数字
decode = lambda l: "".join([itos[i] for i in l]) # 输入一个含有整数的列表，把它解码为字符串

print(encode("Hello, world"))
print(decode(encode("Hello, world")))

[20, 43, 50, 50, 53, 6, 1, 61, 53, 56, 50, 42]
Hello, world


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# 将数据划分为测试集和验证集
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8 # 每次用于训练的文本块大小
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
torch.manual_seed(1337)
batch_size = 4 # 表示有多少个序列会被并行处理
block_size = 8 # 表示上下文的最大长度

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # 生成 batch_size 个随机索引，用来表示抽取的 block 的起始索引
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets")
print(yb.shape)
print(yb)

print("-----")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is 1
when input is [44, 53, 56, 1] the target is 58
when input is [44, 53, 56, 1, 58] the target i

In [11]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # 需要将维度进行压缩，原来是 B*T*C，为了方便反向传播，需要修改为 B*T, C，同时，targets 需要修改为 B*T,1    
            loss = F.cross_entropy(logits, targets) # 因为这是多分类问题，所以需要使用 cross_entropy
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # B 并行处理的句子数量，T 句子的长度，C 词表的大小
        # idx: (B, T)
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # 此时得到的 logits 是 (B, T, C)
            # 但是我们只关心最后一个字符，-1 表示看到最后一个字符时，对下一个字符的预测
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1) # dim=-1 表示对最后一个维度，也就是 vocab 这个维度进行 softmax，得到输出每个字符的概率
            idx_next = torch.multinomial(probs, num_samples=1) # 只抽一个样本
            idx = torch.cat((idx, idx_next), dim=1) # 把新生成的和之前输入的拼接在一起，维度为 (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) # 传入训练数据，调用 forward
print(logits.shape)
print(loss)

idx = torch.ones((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist())) # 第一个 batch

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
 -KIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [13]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

KeyboardInterrupt: 

In [ ]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch("train")
    
    # evaluate the loss
    # 训练的一般流程：计算损失，清空梯度，反向传播，更新参数
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=None) # 清空梯度
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.3798983097076416


In [ ]:
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist())) # 第一个 batch

 be wime d no,
G ble d wsesimeanofirikicave w tco the ndveplk lleee l,
Hingleeg my d ay hy, sowe,
Du clgod SERYORDIst m h ueraren aperemalyotld
s.
LAswhere mou lun heraturt IShe ooounite thelleas, or knde prs
Wisprendy twisorowis maloves ume y t manondes SI'les, fanksdile cad n der oncohe weet o fathircato g w, hif iresorome

But.
TII ton belll inged, iem ADr LLet foursighoof the suco a n Le m auchis m EShalsw man n:
ENThie s acof fod isexfrsthusur ary wor s fe E is n his tath, sealouequr winilll


In [ ]:
torch.manual_seed(1337)
B,T,C = 4, 8, 2
x = torch.randn((B,T,C))
x.shape

NameError: name 'torch' is not defined